In [1]:
# !pip install --upgrade pip
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install diffusers transformers datasets
!pip install --upgrade accelerate  # Ensure latest accelerate version
!pip install git+https://github.com/huggingface/diffusers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [2]:
!git clone https://github.com/huggingface/diffusers
!cd diffusers
!pip install -r /kaggle/working/diffusers/examples/textual_inversion/requirements.txt

Cloning into 'diffusers'...
remote: Enumerating objects: 90215, done.
remote: Counting objects: 100% (882/882), done.
remote: Compressing objects: 100% (460/460), done.
remote: Total 90215 (delta 692), reused 415 (delta 412), pack-reused 89333 (from 4)
Receiving objects: 100% (90215/90215), 67.88 MiB | 19.88 MiB/s, done.
Resolving deltas: 100% (65834/65834), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00


In [3]:
import torch
import torchvision
import torchaudio
print(f'torch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')
print(f'torchaudio version: {torchaudio.__version__}')

torch version: 2.5.1+cu124
torchvision version: 0.20.1+cu124
torchaudio version: 2.5.1+cu124


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

DATA_DIR = "/kaggle/input/itay-dataset"
MODEL_NAME = "stable-diffusion-v1-5/stable-diffusion-v1-5"
OUTPUT_DIR = "/kaggle/working/textual_inversion_output"

Using device: cuda


In [6]:
import yaml
accelerate_config = {
    "compute_environment": "LOCAL_MACHINE",
    "distributed_type": "MULTI_GPU",
    "num_processes": 1,
    "gpu_ids": "all",
    "mixed_precision": "no",
    "use_cpu": False,
    "num_machines": 1,
    "machine_rank": 0,
    "main_training_function": "main",
}
with open("/kaggle/working/accelerate_config.yaml", "w") as f:
    yaml.dump(accelerate_config, f)

!cat /kaggle/working/accelerate_config.yaml

compute_environment: LOCAL_MACHINE
distributed_type: MULTI_GPU
gpu_ids: all
machine_rank: 0
main_training_function: main
mixed_precision: 'no'
num_machines: 1
num_processes: 1
use_cpu: false


In [7]:
training_script = f"""
#!/bin/bash
export MODEL_NAME="{MODEL_NAME}"
export DATA_DIR="{DATA_DIR}"
export OUTPUT_DIR="{OUTPUT_DIR}"

accelerate launch --config_file=/kaggle/working/accelerate_config.yaml \\
  /kaggle/working/diffusers/examples/textual_inversion/textual_inversion.py \\
  --pretrained_model_name_or_path=$MODEL_NAME \\
  --train_data_dir=$DATA_DIR \\
  --learnable_property="object" \\
  --placeholder_token="<bearded-man>" \\
  --initializer_token="man" \\
  --resolution=512 \\
  --train_batch_size=1 \\
  --gradient_accumulation_steps=16 \\
  --max_train_steps=700 \\
  --learning_rate=5.0e-04 \\
  --scale_lr \\
  --lr_scheduler="constant" \\
  --lr_warmup_steps=0 \\
  --output_dir=$OUTPUT_DIR \\
  --gradient_checkpointing \\
  --mixed_precision="no"
"""

In [8]:
with open("/kaggle/working/run_textual_inversion.sh", "w") as f:
    f.write(training_script)

!chmod +x /kaggle/working/run_textual_inversion.sh

!bash /kaggle/working/run_textual_inversion.sh

2025-04-22 04:32:48.377347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745296368.557322     122 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745296368.611393     122 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
tokenizer_config.json: 100%|███████████████████| 806/806 [00:00<00:00, 4.84MB/s]
vocab.json: 100%|██████████████████████████| 1.06M/1.06M [00:00<00:00, 1.75MB/s]
merges.txt: 100%|████████████████████████████| 525k/525k [00:00<00:00, 9.50MB/s]
scheduler_config.json: 100%|███████████████████| 308/308 [00:00<00:00, 1.92MB/s]
{'prediction_type', 'clip_sample_range', 'variance_type', 'rescale_betas_zero_snr', 'thresholding', 'sample_max_v

In [ ]:
from diffusers import DiffusionPipeline
import os.path

embed_path = f"{OUTPUT_DIR}/learned_embeds.safetensors"

if not os.path.exists(embed_path):
    print(f"Lỗi: File {embed_path} không tồn tại")
else:
    pipe = DiffusionPipeline.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        use_safetensors=True
    ).to(DEVICE)

    pipe.load_textual_inversion(embed_path)

In [ ]:
prompt = "analog style a bearded man old money aesthetic, highly detailed,  high-definition, sharp focus, intricate details, professionally retouched, 8k high definition, photorealistic, detailed face,  insanely detailed hair."
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
image.save("/kaggle/working/output_image_2.png")